## Imports

In [1]:
import pandas as pd

from sklearn import preprocessing
from collections import deque
import numpy as np
import random
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
# pt CuDNNLSTM look at this https://stackoverflow.com/questions/60468385/is-there-cudnnlstm-or-cudnngru-alternative-in-tensorflow-2-0

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

The next steps are as follow:
### 1. create the main dataframe
### 2. load the data

In [2]:
main_df = pd.DataFrame()

ratios = ["BCH-USD", "BTC-USD", "ETH-USD", "LTC-USD"]

for ratio in ratios:
    dataset = pd.read_csv(
        f"crypto_data/{ratio}.csv",
        names=["time", "low", "high", "open", "close", "volume"],
    )
    # we only need the close and volume
    dataset.rename(
        columns={"close": f"{ratio}-close", "volume": f"{ratio}-volume"}, inplace=True
    )
    # print(dataset.head())

    # set the time as the index
    dataset.set_index("time", inplace=True)

    # drop the ones we don't need
    dataset.drop(["low", "high", "open"], axis=1, inplace=True)

    # merge all the columns on the index
    if main_df.empty:
        main_df = dataset
    else:
        main_df = main_df.join(dataset)

In [3]:
main_df.head()

,BCH-USD-close,BCH-USD-volume,BTC-USD-close,BTC-USD-volume,ETH-USD-close,ETH-USD-volume,LTC-USD-close,LTC-USD-volume
time,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978


In [4]:
main_df.columns.values

array(['BCH-USD-close', 'BCH-USD-volume', 'BTC-USD-close',
       'BTC-USD-volume', 'ETH-USD-close', 'ETH-USD-volume',
       'LTC-USD-close', 'LTC-USD-volume'], dtype=object)

## Define the parameters of the dataset.

In [5]:
SEQ_LEN = 60
FUTURE_PRED = 60
COIN = "BTC-USD"
VAL_PCT = 0.20

## Create a classifier that specifies when to buy/sell


In [6]:
def classify(current, future):
    if float(current) <= float(future):
        return 1 # when the price is higher we buy
    else:
        return 0 # when the price is lower we sell

## Let's do some feature engineering
- add a 'future' column
- add a 'target' column

In [7]:
main_df["future"] = main_df[f"{COIN}-close"].shift(-FUTURE_PRED)
main_df[[f'{COIN}-close', 'future']].head()

,BTC-USD-close,future
time,,
1528968660,6489.549805,6485.000000
1528968720,6487.379883,6484.990234
1528968780,6479.410156,6493.399902
1528968840,6479.410156,6487.830078
1528968900,6479.979980,6487.830078


In [8]:
main_df["target"] = list(
    map(classify, main_df[f"{COIN}-close"], main_df["future"])
)
main_df.head()

,BCH-USD-close,BCH-USD-volume,BTC-USD-close,BTC-USD-volume,ETH-USD-close,ETH-USD-volume,LTC-USD-close,LTC-USD-volume,future,target
time,,,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200,6485.000000,0
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024,6484.990234,0
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799,6493.399902,1
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067,6487.830078,1
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978,6487.830078,1


## Split the data into training and validation sets

In [9]:
times = main_df.index.values
last_x_pct = main_df.index.values[-int(VAL_PCT * len(times))]

validation_main_df = main_df[main_df.index >= last_x_pct]
main_df = main_df[main_df.index < last_x_pct]

main_df.loc[main_df.index == last_x_pct] #the splitting place

,BCH-USD-close,BCH-USD-volume,BTC-USD-close,BTC-USD-volume,ETH-USD-close,ETH-USD-volume,LTC-USD-close,LTC-USD-volume,future,target
time,,,,,,,,,,


In [10]:
main_df.describe()

,BCH-USD-close,BCH-USD-volume,BTC-USD-close,BTC-USD-volume,ETH-USD-close,ETH-USD-volume,LTC-USD-close,LTC-USD-volume,future,target
count,73780.000000,73780.000000,68659.000000,68659.000000,73625.000000,73625.000000,73107.000000,73107.000000,68659.000000,73780.000000
mean,762.449577,11.751972,6868.055547,6.258522,455.072312,49.440810,82.332467,118.398840,6867.720904,0.462591
std,73.228726,34.029166,682.278698,16.738563,38.534674,129.805149,8.303777,272.419967,682.557930,0.498602
min,559.479980,0.000002,5778.109863,0.001915,326.320007,0.000030,58.290001,0.009593,5778.109863,0.000000
25%,708.580017,0.420771,6350.584961,0.864259,434.299988,5.499268,77.279999,12.922469,6350.109863,0.000000
50%,759.000000,2.020262,6635.700195,2.077186,461.700012,14.897300,82.339996,35.887356,6635.700195,0.000000
75%,819.140015,9.129493,7415.069824,5.135763,475.200012,42.180565,86.150002,106.874336,7415.069824,1.000000
max,927.000000,1520.833862,8482.799805,471.660889,547.000000,9310.024414,103.040001,10263.191406,8482.799805,1.000000


In [11]:
missing_target_values_pct = main_df['future'].isna().sum() * 100 /len(main_df['future'])
print(f'{missing_target_values_pct:0.3}%')

6.94%


There are 6% missing values in the target column alone.

In [12]:
main_df.isna().sum()

BCH-USD-close        0
BCH-USD-volume       0
BTC-USD-close     5121
BTC-USD-volume    5121
ETH-USD-close      155
ETH-USD-volume     155
LTC-USD-close      673
LTC-USD-volume     673
future            5121
target               0
dtype: int64

In [13]:
main_df.shape

(73780, 10)

## We now remove/fill in the missing values with values that we believe to be correct.

In [14]:
main_df.fillna(method="ffill", inplace=True)
validation_main_df.fillna(method="ffill", inplace=True)

main_df.dropna(inplace=True)
validation_main_df.dropna(inplace=True)
main_df.isna().sum(), validation_main_df.isna().sum()

(BCH-USD-close     0
 BCH-USD-volume    0
 BTC-USD-close     0
 BTC-USD-volume    0
 ETH-USD-close     0
 ETH-USD-volume    0
 LTC-USD-close     0
 LTC-USD-volume    0
 future            0
 target            0
 dtype: int64,
 BCH-USD-close     0
 BCH-USD-volume    0
 BTC-USD-close     0
 BTC-USD-volume    0
 ETH-USD-close     0
 ETH-USD-volume    0
 LTC-USD-close     0
 LTC-USD-volume    0
 future            0
 target            0
 dtype: int64)

### The data is still not ready for processing. There is one last step required: preprocessing.
# Normalizing, Scaling and Balancing the dataset

The steps required are: dropping the future column, creating pct_change so that the data is clearer and normalizare + scale the data.

# SEQ_LEN
With a normal dataset, we would randomize the order in which the data is so that the model doesn't overfit; instead it learns on random data.

We can't do that here cuz the data would be too random and the model can't learn. Instead, what we'll do is take sequences of predefined lengths then shuffle those sequences. It's a really smart thing to do. We get continuous 1 hour (60 mins) periods and then shuffle them around.

In [15]:
def preprocessing_df(df):
    df.drop("future", axis=1)  # we drop the future column

    # use pct change to figure the change in price
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)

            # normalize the data 0 -> 1 NOTE: normalization is a form of scaling
            df[col] = preprocessing.scale(df[col])
    df.dropna(inplace=True)
    sequencial_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for row in df.values:
        prev_days.append([x for x in row[:-1]])
        if len(prev_days) == SEQ_LEN:
            #separate the label/target
            sequencial_data.append([np.array(prev_days), row[-1]])

    random.shuffle(sequencial_data)

    """
    the last step in our preprocessing will be to balance the learning set
    """
    buys = []
    sells = []

    for seq, target in sequencial_data:
        if target == 0: # you need to sell
            sells.append([seq, target])
        elif target == 1: # you need to buy
            buys.append([seq, target])

    random.shuffle(buys)
    random.shuffle(sells)

    # how do we balance? Simple. we remove the excess

    lower_nr = min(len(buys), len(sells))

    buys = buys[:lower_nr]
    sells = sells[:lower_nr]

    sequencial_data = buys + sells

    random.shuffle(sequencial_data)

    # split into x and Y
    x = []
    Y = []

    for seq, target in sequencial_data:
        x.append(seq)
        Y.append(target)
    return np.array(x), np.array(Y)

In [16]:
train_x, train_Y = preprocessing_df(main_df)
test_x, test_Y = preprocessing_df(validation_main_df)

In [17]:
print(f"Training data: {len(train_x)}, Validation data: {len(test_x)}")
print(f" Training set Sells: {(train_Y.astype('int32') == 0).sum()}, Buys: {(train_Y.astype('int32') == 1).sum()}")
print(f" Validation set Sells: {(test_Y.astype('int32') == 0).sum()}, Buys: {(test_Y.astype('int32') == 1).sum()}")

Training data: 68188, Validation data: 17404
 Training set Sells: 34094, Buys: 34094
 Validation set Sells: 8702, Buys: 8702


As we can see, the total of entries is 68.010 + 17.120 = 85130 (after filling/dropping the missing values) and the ratio is 80/20.

# Now that the preprocessing is complete, we can go and train the model.

In [18]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

model = Sequential()
model.add(LSTM(64, kernel_regularizer=l1(1e-6), recurrent_regularizer=l1(1e-6), bias_regularizer=l1(1e-6), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(64, kernel_regularizer=l1(1e-6), recurrent_regularizer=l1(1e-6), bias_regularizer=l1(1e-6), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(64, kernel_regularizer=l1(1e-6), recurrent_regularizer=l1(1e-6), bias_regularizer=l1(1e-6)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [19]:
opt = Adam(learning_rate=1e-4, decay=1e-7)

# lr = ExponentialDecay(initial_learning_rate=1e-2, decay_steps=10, decay_rate=.1)
# opt = SGD(learning_rate=lr)
LOSS='sparse_categorical_crossentropy'

model.compile(loss=LOSS, optimizer=opt, metrics=['accuracy'])

## Hyperparameters, tensorboard and checkpoint

In [20]:
EPOCHS = 10
BATCH_SIZE = 32
NAME = f"{COIN}-{SEQ_LEN}-{FUTURE_PRED}-VALID-PCT-{VAL_PCT}-LOSS-{LOSS}-OPT-{'Adam'}-{int(time.time())}"
# NAME = f"{COIN}-{SEQ_LEN}-{FUTURE_PRED}-VALID-PCT-{VAL_PCT}-LOSS-{LOSS}-OPT-{'SGD'}-{int(time.time())}"
# NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

In [21]:
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

# Fitting the model

In [22]:
history = model.fit(train_x, train_Y, 
        batch_size=BATCH_SIZE, 
        epochs=EPOCHS,
        validation_data=(test_x, test_Y),
        # callbacks=[tensorboard, checkpoint])
        callbacks=[tensorboard])

Epoch 1/10
2131/2131 [==============================] - 83s 34ms/step - loss: 0.5111 - accuracy: 0.7423 - val_loss: 0.3488 - val_accuracy: 0.8659
Epoch 2/10
2131/2131 [==============================] - 75s 35ms/step - loss: 0.1937 - accuracy: 0.9259 - val_loss: 0.2066 - val_accuracy: 0.9166
Epoch 3/10
2131/2131 [==============================] - 75s 35ms/step - loss: 0.1534 - accuracy: 0.9414 - val_loss: 0.2735 - val_accuracy: 0.8970
Epoch 4/10
2131/2131 [==============================] - 78s 37ms/step - loss: 0.1338 - accuracy: 0.9503 - val_loss: 0.2016 - val_accuracy: 0.9312
Epoch 5/10
2131/2131 [==============================] - 72s 34ms/step - loss: 0.1217 - accuracy: 0.9543 - val_loss: 0.1818 - val_accuracy: 0.9381
Epoch 6/10
2131/2131 [==============================] - 79s 37ms/step - loss: 0.1082 - accuracy: 0.9608 - val_loss: 0.1940 - val_accuracy: 0.9329
Epoch 7/10
2131/2131 [==============================] - 79s 37ms/step - loss: 0.1051 - accuracy: 0.9613 - val_loss: 0.2245 -

# Score and saving the model

In [23]:
# Scoring
score = model.evaluate(test_x, test_Y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Saving
model.save(f"models/{NAME}")

Test loss: 0.23146085441112518
Test accuracy: 0.9384049773216248
INFO:tensorflow:Assets written to: models/BTC-USD-60-60-VALID-PCT-0.2-LOSS-sparse_categorical_crossentropy-OPT-Adam-1614605606\assets
INFO:tensorflow:Assets written to: models/BTC-USD-60-60-VALID-PCT-0.2-LOSS-sparse_categorical_crossentropy-OPT-Adam-1614605606\assets


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 64)            18944     
_________________________________________________________________
dropout (Dropout)            (None, 60, 64)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 64)            256       
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 64)            33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 64)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 64)            256       
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                3